# Setup

In [114]:
%pip install scikit-learn

You should consider upgrading via the 'c:\Users\kavan\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [115]:
%pip install wandb

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\kavan\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [164]:
import wandb
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
import numpy as np

# Data

In [129]:
data = load_breast_cancer()
print('type:', type(data))
print('keys:', data.keys())
print('shape:', data.data.shape)

type: <class 'sklearn.utils._bunch.Bunch'>
keys: dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
shape: (569, 30)


In [130]:
print('ground truth:', data.target)
print('classes:', data.target_names)
print('features:', data.feature_names)

ground truth: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 0
 1 0 1 1 1 0 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 0 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 

In [131]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [132]:
# Split the dataset into a train and test set. Pass in feature data and targets
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33)
N, D = X_train.shape

In [133]:
# Scale dataset for comparable weights in features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [186]:
wandb.init(project='tensorflow-test', entity='kavp')

# WandB parameters
wandb.config = {
    'learning_rate': 0.001,
    'epochs': 100,
    'batch_size': 128,
    'eager_mode': False,
}
if wandb.config['eager_mode'] == True:
    tf.compat.v1.enable_eager_execution()
elif wandb.config['eager_mode'] == False:
    tf.compat.v1.disable_eager_execution()
else:
    raise ValueError('eager_mode property of wandb config could not be determined.') 


# Define our own sequential model
model = tf.keras.models.Sequential([
    # Layers
    tf.keras.layers.Input(shape=(D,)),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
    run_eagerly=wandb.config['eager_mode'],
)

In [187]:
with tf.compat.v1.Session() as sess:
    r = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=wandb.config['epochs'], batch_size=wandb.config['batch_size'], )
    wandb.tensorflow.log(tf.compat.v1.summary.merge_all())
    wandb.log({'loss': r.history['loss'], 'val_loss': r.history['val_loss'], 'acc': r.history['accuracy'], 'val_acc': r.history['val_accuracy']})

Train on 381 samples, validate on 188 samples
Epoch 1/100
381/381 [==============================] - 0s 574us/sample - loss: 1.7826 - accuracy: 0.0709 - val_loss: 1.6698 - val_accuracy: 0.1170
Epoch 2/100
381/381 [==============================] - 0s 13us/sample - loss: 1.7491 - accuracy: 0.0735 - val_loss: 1.6384 - val_accuracy: 0.1223
Epoch 3/100
381/381 [==============================] - 0s 14us/sample - loss: 1.7153 - accuracy: 0.0735 - val_loss: 1.6075 - val_accuracy: 0.1170
Epoch 4/100
381/381 [==============================] - 0s 13us/sample - loss: 1.6836 - accuracy: 0.0814 - val_loss: 1.5767 - val_accuracy: 0.1223
Epoch 5/100
381/381 [==============================] - 0s 11us/sample - loss: 1.6499 - accuracy: 0.0945 - val_loss: 1.5464 - val_accuracy: 0.1223
Epoch 6/100
381/381 [==============================] - 0s 12us/sample - loss: 1.6182 - accuracy: 0.1024 - val_loss: 1.5164 - val_accuracy: 0.1170
Epoch 7/100
381/381 [==============================] - 0s 11us/sample - loss:

In [188]:
wandb_data = [[x,y] for (x,y) in zip(np.arange(0, wandb.config['epochs'], 1), r.history['loss'])]
table = wandb.Table(data=wandb_data, columns = ["epoch", "loss"])
wandb.log({"loss_against_epochs" : wandb.plot.line(table, "epoch", "loss", title="Training loss")})

In [189]:
wandb_data = [[x,y] for (x,y) in zip(np.arange(0, wandb.config['epochs'], 1), r.history['val_loss'])]
table = wandb.Table(data=wandb_data, columns = ["epoch", "val_loss"])
wandb.log({"val_loss_against_epochs" : wandb.plot.line(table, "epoch", "val_loss", title="Validation loss")})

In [190]:
wandb_data = [[x,y] for (x,y) in zip(np.arange(0, wandb.config['epochs'], 1), r.history['accuracy'])]
table = wandb.Table(data=wandb_data, columns = ["epoch", "accuracy"])
wandb.log({"accuracy_against_epochs" : wandb.plot.line(table, "epoch", "accuracy", title="Training accuracy")})

In [191]:
wandb_data = [[x,y] for (x,y) in zip(np.arange(0, wandb.config['epochs'], 1), r.history['val_accuracy'])]
table = wandb.Table(data=wandb_data, columns = ["epoch", "val_accuracy"])
wandb.log({"val_accuracy_against_epochs" : wandb.plot.line(table, "epoch", "val_accuracy", title="Validation accuracy")})

In [192]:
wandb.finish()